<a href="https://colab.research.google.com/github/Nurudeen2020/Python-for-Machine-Learning/blob/main/Optimized_MNIST_Fashion_CNN_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install keras-tuner


     |████████████████████████████████| 71kB 4.1MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp36-none-any.whl size=78939 sha256=1b152a80c40fc06ab734152d909f3189cf61f4cd84e8642b08bdbb5731b20dc8
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15358 sha256=56b9923025ea9236d625fd5161a5e80b5d084d1305399f34cbf66564c6a777f6
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [4]:
import numpy as np
import tensorflow as tf
from tensorflow import keras 

In [5]:
print(tf.__version__)

2.4.0


In [6]:
#loading the fashion mnsist datasets

In [7]:
fashion_mnist= keras.datasets.fashion_mnist

In [8]:
(train_images, train_labels), (test_images, test_labels)= fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [9]:
train_images= train_images/255.0
test_images= test_images/255.0

In [10]:
train_images[0]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.    

In [11]:
train_images[0].shape

(28, 28)

In [12]:
train_images= train_images.reshape(len(train_images),28,28,1)
test_images= test_images.reshape(len(test_images),28,28,1)

In [13]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [14]:
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [16]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [18]:
tuner_search.search(train_images, train_labels, epochs=3, validation_split=0.1)

Trial 6 Complete [00h 00m 16s]
val_accuracy: 0.8485000133514404

Best val_accuracy So Far: 0.9131666421890259
Total elapsed time: 00h 02m 25s
INFO:tensorflow:Oracle triggered exit


In [19]:
model=tuner_search.get_best_models(num_models=1)[0]

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 128)       1280      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 48)        55344     
_________________________________________________________________
flatten (Flatten)            (None, 27648)             0         
_________________________________________________________________
dense (Dense)                (None, 80)                2211920   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                810       
Total params: 2,269,354
Trainable params: 2,269,354
Non-trainable params: 0
_________________________________________________________________


In [21]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.1668 - accuracy: 0.9387 - val_loss: 0.2526 - val_accuracy: 0.9135
Epoch 5/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.1160 - accuracy: 0.9581 - val_loss: 0.2504 - val_accuracy: 0.9145
Epoch 6/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0756 - accuracy: 0.9723 - val_loss: 0.2986 - val_accuracy: 0.9147
Epoch 7/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0505 - accuracy: 0.9813 - val_loss: 0.3289 - val_accuracy: 0.9132
Epoch 8/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0346 - accuracy: 0.9879 - val_loss: 0.4088 - val_accuracy: 0.9098
Epoch 9/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0267 - accuracy: 0.9906 - val_loss: 0.4893 - val_accuracy: 0.9102
Epoch 10/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0214 - accuracy: 0.9924 - val_loss: 0.5062 - val_accuracy